<a href="https://colab.research.google.com/github/nadiajelani/wound-detection/blob/main/Wound_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install tensorflow pillow numpy

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

def train_wound_cnn_model(data_dir, save_path="wound_cnn_model.h5"):
    # Data augmentation and normalization
    datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

    # Training and validation data generators
    train_generator = datagen.flow_from_directory(
        data_dir,
        target_size=(128, 128),
        batch_size=32,
        class_mode='binary',
        subset='training'
    )
    val_generator = datagen.flow_from_directory(
        data_dir,
        target_size=(128, 128),
        batch_size=32,
        class_mode='binary',
        subset='validation'
    )

    # Debugging dataset loading
    if train_generator.samples == 0:
        raise ValueError("No training images found. Check your dataset structure and path.")
    if val_generator.samples == 0:
        raise ValueError("No validation images found. Check your dataset structure and path.")

    # CNN Model
    model = Sequential([
        Input(shape=(128, 128, 3)),  # Define input shape explicitly
        Conv2D(32, (3, 3), activation='relu'),
        MaxPooling2D(pool_size=(2, 2)),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D(pool_size=(2, 2)),
        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(1, activation='sigmoid')  # Binary classification: infected/non-infected
    ])

    # Compile the model
    model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])

    # Train the model
    model.fit(
        train_generator,
        validation_data=val_generator,
        epochs=10
    )

    # Save the model
    model.save(save_path)
    print(f"Model saved to {save_path}")

# Train and save the model
train_wound_cnn_model("/content/drive/MyDrive/Colab Notebooks/Jay Jelani/dataset", "wound_cnn_model.h5")


Found 0 images belonging to 3 classes.
Found 0 images belonging to 3 classes.


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


ValueError: Must provide at least one structure

In [ ]:
import os
import subprocess
import numpy as np
from PIL import Image, ImageEnhance, ImageDraw
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.metrics import BinaryAccuracy
from tkinter import Tk, filedialog, Button
from datetime import datetime

In [ ]:
# Load the ResNet50 model for feature extraction
base_model = ResNet50(weights='imagenet')
model = Model(inputs=base_model.input, outputs=base_model.get_layer('avg_pool').output)

# Load the U-Net model for segmentation
try:
    unet_model = load_model('/content/drive/MyDrive/Colab Notebooks/Jay Jelani/unet_model.h5')
except IOError:
    print("Error: U-Net model file could not be loaded. Check the file path.")
    unet_model = None

# Load the CNN model for wound classification and compile it
try:
    cnn_model = load_model('/Users/nadiajelani/Desktop/Medical Proj/models/wound_cnn_model.h5')
    cnn_model.compile(loss=BinaryCrossentropy(), metrics=[BinaryAccuracy()])
except IOError:
    print("Error: CNN model file could not be loaded. Check the file path.")
    cnn_model = None

# Global variables to control capturing and directory path
is_capturing = True
save_directory = ""
latest_image_path = ""


102967424/102967424 ━━━━━━━━━━━━━━━━━━━━ 3s 0us/step
Error: U-Net model file could not be loaded. Check the file path.
Error: CNN model file could not be loaded. Check the file path.


In [ ]:
def ask_save_directory():
    return filedialog.askdirectory(title="Select Save Directory")

def enhance_image(image_path):
    img = Image.open(image_path)
    enhancer = ImageEnhance.Detail(img)
    img_enhanced = enhancer.enhance(1.5)  # Adjust enhancement level as needed
    img_enhanced.save(image_path)
    print(f"Image enhanced and saved to {image_path}")

def segment_image_with_unet(image_path):
    if unet_model is None:
        print("U-Net model is not loaded. Segmentation cannot proceed.")
        return None

    img = Image.open(image_path).resize((128, 128))  # resize to model input size
    img_array = np.array(img) / 255.0  # normalize
    img_array = np.expand_dims(img_array, axis=0)  # add batch dimension
    mask = unet_model.predict(img_array)[0]  # assume unet_model works on this shape
    binary_mask = (mask > 0.5).astype(np.uint8) * 255  # convert to binary
    return Image.fromarray(binary_mask.squeeze(), 'L')


In [ ]:
def classify_wound_area(image, bounding_box):
    if cnn_model is None:
        print("CNN model is not loaded. Classification cannot proceed.")
        return "Unknown", 0.0

    x, y, w, h = bounding_box
    wound_region = image.crop((x, y, x + w, y + h))
    wound_region = wound_region.resize((128, 128))  # Adjust to CNN model input size if different
    wound_array = np.array(wound_region) / 255.0  # Normalize
    wound_array = np.expand_dims(wound_array, axis=0)  # Add batch dimension

    # Classify wound area using CNN model
    classification = cnn_model.predict(wound_array)
    label = "Infected" if classification[0][0] > 0.5 else "Non-Infected"
    confidence = classification[0][0] if label == "Infected" else 1 - classification[0][0]

    return label, confidence


In [ ]:
def segment_and_label_wound(image_path):
    img = Image.open(image_path)
    mask_image = segment_image_with_unet(image_path)

    if mask_image is None:
        print("Segmentation could not be completed.")
        return None, None

    # Resize mask to original image size
    mask_resized = mask_image.resize(img.size)
    mask_np = np.array(mask_resized)

    labeled_image = img.copy()
    draw = ImageDraw.Draw(labeled_image)
    wound_areas = []

    # Dummy bounding boxes for demonstration (replace with real detection as needed)
    for i in range(0, img.width, 100):
        for j in range(0, img.height, 100):
            x, y, w, h = i, j, 50, 50
            draw.rectangle([(x, y), (x + w, y + h)], outline="green", width=2)

            # Classify wound area
            label, confidence = classify_wound_area(img, (x, y, w, h))
            draw.text((x, y - 10), f"{label} ({confidence:.2f})", fill="green")
            wound_areas.append((x, y, w, h, label, confidence))

    labeled_image_path = image_path.replace('.jpg', '_labeled.jpg')
    labeled_image.save(labeled_image_path)
    print(f"Labeled image saved to {labeled_image_path}")

    return labeled_image_path, wound_areas


In [ ]:
def export_report(image_path, wound_areas):
    report_path = image_path.replace('.jpg', '_report.txt')
    with open(report_path, 'w') as report:
        report.write(f"Wound Analysis Report - {datetime.now()}\n")
        report.write(f"Image Path: {image_path}\n")
        report.write("Detected Wound Areas (in pixels):\n")
        for area in wound_areas:
            x, y, w, h, label, confidence = area
            report.write(f"- Position: ({x}, {y}), Size: ({w}x{h}), Label: {label}, Confidence: {confidence:.2f}\n")
        total_area = sum(area[2] * area[3] for area in wound_areas)
        report.write(f"Total Wound Area: {total_area} pixels\n")
    print(f"Report saved to {report_path}")


In [ ]:
def capture_image():
    global latest_image_path
    if is_capturing and save_directory:
        image_number = len(os.listdir(save_directory)) + 1
        latest_image_path = os.path.join(save_directory, f'captured_image_{image_number}.jpg')

        # Capture the image using imagesnap
        subprocess.run(['imagesnap', latest_image_path])

        # Enhance the captured image before saving
        enhance_image(latest_image_path)

        print(f"Image saved to {latest_image_path}")


In [ ]:
def button_click(x, y):
    global is_capturing, save_directory, latest_image_path
    if -50 <= x <= 50 and -25 <= y <= 25:
        if not save_directory:
            save_directory = ask_save_directory()
        capture_image()
    elif -50 <= x <= 50 and -75 <= y <= -25:
        is_capturing = False
        print("Image capturing stopped.")
    elif -50 <= x <= 50 and -125 <= y <= -75:
        if latest_image_path:
            labeled_image_path, wound_areas = segment_and_label_wound(latest_image_path)
            if labeled_image_path and wound_areas:
                export_report(labeled_image_path, wound_areas)
        else:
            print("No image captured to analyze.")


In [ ]:
def create_gui():
    root = Tk()
    root.title("Wound Detection Tool")

    capture_btn = Button(root, text="Capture Image", command=lambda: button_click(0, 0))
    capture_btn.pack(pady=10)

    stop_btn = Button(root, text="Stop Capturing", command=lambda: button_click(0, -50))
    stop_btn.pack(pady=10)

    analyze_btn = Button(root, text="Analyze Wound", command=lambda: button_click(0, -100))
    analyze_btn.pack(pady=10)

    exit_btn = Button(root, text="Exit", command=root.quit)
    exit_btn.pack(pady=10)

    root.mainloop()

# Start the GUI
create_gui()


In [ ]:
import os
import subprocess
import numpy as np
from PIL import Image, ImageEnhance, ImageDraw
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.metrics import BinaryAccuracy
from tkinter import Tk, filedialog, Button
from datetime import datetime

# Load the ResNet50 model for feature extraction
try:
    base_model = ResNet50(weights='imagenet')
    model = Model(inputs=base_model.input, outputs=base_model.get_layer('avg_pool').output)
except Exception as e:
    print(f"Error loading ResNet50 model: {e}")
    model = None

# Load the U-Net model for segmentation
try:
    unet_model = load_model('/Users/nadiajelani/Desktop/Medical Proj/models/unet_model.h5')
except IOError:
    print("Error: U-Net model file could not be loaded. Check the file path.")
    unet_model = None

# Load the CNN model for wound classification and compile it
try:
    cnn_model = load_model('/Users/nadiajelani/Desktop/Medical Proj/models/wound_cnn_model.h5')
    cnn_model.compile(loss=BinaryCrossentropy(), metrics=[BinaryAccuracy()])
except IOError:
    print("Error: CNN model file could not be loaded. Check the file path.")
    cnn_model = None

# Global variables to control capturing and directory path
is_capturing = True
save_directory = ""
latest_image_path = ""

def ask_save_directory():
    return filedialog.askdirectory(title="Select Save Directory")

def enhance_image(image_path):
    try:
        img = Image.open(image_path)
        enhancer = ImageEnhance.Detail(img)
        img_enhanced = enhancer.enhance(1.5)  # Adjust enhancement level as needed
        img_enhanced.save(image_path)
        print(f"Image enhanced and saved to {image_path}")
    except Exception as e:
        print(f"Error enhancing image: {e}")

def segment_image_with_unet(image_path):
    if unet_model is None:
        print("U-Net model is not loaded. Segmentation cannot proceed.")
        return None

    try:
        img = Image.open(image_path).resize((128, 128))  # resize to model input size
        img_array = np.array(img) / 255.0  # normalize
        img_array = np.expand_dims(img_array, axis=0)  # add batch dimension
        mask = unet_model.predict(img_array)[0]  # assume unet_model works on this shape
        binary_mask = (mask > 0.5).astype(np.uint8) * 255  # convert to binary
        return Image.fromarray(binary_mask.squeeze(), 'L')
    except Exception as e:
        print(f"Error during segmentation: {e}")
        return None

def classify_wound_area(image, bounding_box):
    if cnn_model is None:
        print("CNN model is not loaded. Classification cannot proceed.")
        return "Unknown", 0.0

    try:
        x, y, w, h = bounding_box
        wound_region = image.crop((x, y, x + w, y + h))
        wound_region = wound_region.resize((128, 128))  # Adjust to CNN model input size if different
        wound_array = np.array(wound_region) / 255.0  # Normalize
        wound_array = np.expand_dims(wound_array, axis=0)  # Add batch dimension

        # Classify wound area using CNN model
        classification = cnn_model.predict(wound_array)
        label = "Infected" if classification[0][0] > 0.5 else "Non-Infected"
        confidence = classification[0][0] if label == "Infected" else 1 - classification[0][0]

        return label, confidence
    except Exception as e:
        print(f"Error during classification: {e}")
        return "Unknown", 0.0

def segment_and_label_wound(image_path):
    try:
        img = Image.open(image_path)
        mask_image = segment_image_with_unet(image_path)

        if mask_image is None:
            print("Segmentation could not be completed.")
            return None, None

        # Resize mask to original image size
        mask_resized = mask_image.resize(img.size)
        mask_np = np.array(mask_resized)

        labeled_image = img.copy()
        draw = ImageDraw.Draw(labeled_image)
        wound_areas = []

        # Dummy bounding boxes for demonstration (replace with real detection as needed)
        for i in range(0, img.width, 100):
            for j in range(0, img.height, 100):
                x, y, w, h = i, j, 50, 50
                draw.rectangle([(x, y), (x + w, y + h)], outline="green", width=2)

                # Classify wound area
                label, confidence = classify_wound_area(img, (x, y, w, h))
                draw.text((x, y - 10), f"{label} ({confidence:.2f})", fill="green")
                wound_areas.append((x, y, w, h, label, confidence))

        labeled_image_path = image_path.replace('.jpg', '_labeled.jpg')
        labeled_image.save(labeled_image_path)
        print(f"Labeled image saved to {labeled_image_path}")

        return labeled_image_path, wound_areas
    except Exception as e:
        print(f"Error during segmentation and labeling: {e}")
        return None, None

def export_report(image_path, wound_areas):
    try:
        report_path = image_path.replace('.jpg', '_report.txt')
        with open(report_path, 'w') as report:
            report.write(f"Wound Analysis Report - {datetime.now()}\n")
            report.write(f"Image Path: {image_path}\n")
            report.write("Detected Wound Areas (in pixels):\n")
            for area in wound_areas:
                x, y, w, h, label, confidence = area
                report.write(f"- Position: ({x}, {y}), Size: ({w}x{h}), Label: {label}, Confidence: {confidence:.2f}\n")
            total_area = sum(area[2] * area[3] for area in wound_areas)
            report.write(f"Total Wound Area: {total_area} pixels\n")
        print(f"Report saved to {report_path}")
    except Exception as e:
        print(f"Error exporting report: {e}")

def capture_image():
    global latest_image_path
    if is_capturing and save_directory:
        try:
            image_number = len(os.listdir(save_directory)) + 1
            latest_image_path = os.path.join(save_directory, f'captured_image_{image_number}.jpg')

            # Capture the image using imagesnap
            subprocess.run(['imagesnap', latest_image_path])

            # Enhance the captured image before saving
            enhance_image(latest_image_path)

            print(f"Image saved to {latest_image_path}")
        except Exception as e:
            print(f"Error capturing image: {e}")

def button_click(x, y):
    global is_capturing, save_directory, latest_image_path
    try:
        if -50 <= x <= 50 and -25 <= y <= 25:
            if not save_directory:
                save_directory = ask_save_directory()
            capture_image()
        elif -50 <= x <= 50 and -75 <= y <= -25:
            is_capturing = False
            print("Image capturing stopped.")
        elif -50 <= x <= 50 and -125 <= y <= -75:
            if latest_image_path:
                labeled_image_path, wound_areas = segment_and_label_wound(latest_image_path)
                if labeled_image_path and wound_areas:
                    export_report(labeled_image_path, wound_areas)
            else:
                print("No image captured to analyze.")
    except Exception as e:
        print(f"Error in button click handling: {e}")

def create_gui():
    try:
        root = Tk()
        root.title("Wound Detection Tool")

        capture_btn = Button(root, text="Capture Image", command=lambda: button_click(0, 0))
        capture_btn.pack(pady=10)

        stop_btn = Button(root, text="Stop Capturing", command=lambda: button_click(0, -50))
        stop_btn.pack(pady=10)

        analyze_btn = Button(root, text="Analyze Wound", command=lambda: button_click(0, -100))
        analyze_btn.pack(pady=10)

        exit_btn = Button(root, text="Exit", command=root.quit)
        exit_btn.pack(pady=10)

        root.mainloop()
    except Exception as e:
        print(f"Error creating GUI: {e}")

if __name__ == "__main__":
    print("Starting GUI for Wound Detection Tool...")
    create_gui()
    print("GUI exited.")
